### Solar Farm EDA - Benin (Malanville and Parakou)
### Project: MoonLight Energy Solutions

In [ ]:


# Import Required Libraries
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import plotly.express as px
import plotly.graph_objects as go

# Plot settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


# Data Loading and preparing
# ----------------------------  
# Configuration
DATA_DIR = Path('../data/benin/')  # Adjust path if needed
DATE_FORMAT = "%Y-%m-%d %H:%M"
ENCODINGS = ['latin1', 'iso-8859-1', 'cp1252', 'utf-16']

# Robust CSV reader
def try_read_csv(filepath):
    for enc in ENCODINGS:
        try:
            df = pd.read_csv(
                filepath,
                encoding=enc,
                parse_dates=['Timestamp'],
                date_format=DATE_FORMAT,
                low_memory=False,
                skiprows=[1]
            )
            return df
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Failed to read {filepath} with any of the tried encodings.")

# Load and clean Benin data
def load_and_clean_data():
    print("Loading Benin solar data...")
    
    malanville = try_read_csv(DATA_DIR / 'solar-measurements_benin-malanville_qc.csv')
    parakou = try_read_csv(DATA_DIR / 'solar-measurements_benin-parakou_qc.csv')
    malanville['Location'] = 'Malanville'
    parakou['Location'] = 'Parakou'
    
    benin_df = pd.concat([malanville, parakou], ignore_index=True)
    
    try:
        malanville_y2 = try_read_csv(DATA_DIR / 'solar-measurements_benin-malanville_qc_year2.csv')
        parakou_y2 = try_read_csv(DATA_DIR / 'solar-measurements_benin-parakou_qc_year2.csv')
        malanville_y2['Location'] = 'Malanville'
        parakou_y2['Location'] = 'Parakou'
        benin_df = pd.concat([benin_df, malanville_y2, parakou_y2], ignore_index=True)
        print("Year 2 data loaded successfully.")
    except FileNotFoundError:
        print("Year 2 data not found. Proceeding with Year 1 data only.")
    
    return benin_df

# Load data
benin_df = load_and_clean_data()

# ----------------------------
# Summary Stats & Missing Values
# ----------------------------

print("\nDescriptive statistics:")
display(benin_df.describe())

print("\nMissing value count per column:")
display(benin_df.isna().sum())

print("\nColumns with >5% missing values:")
missing_report = benin_df.isna().mean() * 100
display(missing_report[missing_report > 5])

# ----------------------------
# Outlier Detection
# ----------------------------

cols_to_check = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
temp_df = benin_df[cols_to_check].apply(pd.to_numeric, errors='coerce')
temp_df = temp_df.fillna(temp_df.median())

means = temp_df.mean()
stds = temp_df.std()
z_scores = (temp_df - means) / stds
outliers = (z_scores.abs() > 3).any(axis=1)

print(f"Outliers detected: {outliers.sum()} out of {len(temp_df)} rows")

# ----------------------------
# Data Cleaning
# ----------------------------

df_clean = benin_df.copy()

# Remove "Comments" column if it exists
df_clean.drop(columns=['Comments'], inplace=True, errors='ignore')

# Convert and impute numeric columns
df_clean[cols_to_check] = df_clean[cols_to_check].apply(pd.to_numeric, errors='coerce')
df_clean[cols_to_check] = df_clean[cols_to_check].fillna(df_clean[cols_to_check].median())

# ----------------------------
# Export Cleaned CSV
# ----------------------------

output_path = Path('../data/benin/benin_clean.csv')
df_clean.to_csv(output_path, index=False)
print(f"\nCleaned dataset saved to {output_path}")


#### Time series analysis

In [ ]:
# Ensure 'Tamb' is numeric for plotting
df_clean['Tamb'] = pd.to_numeric(df_clean['Tamb'], errors='coerce')

for col in ['GHI', 'DNI', 'DHI', 'Tamb']:
    plt.figure(figsize=(14, 4))
    sns.lineplot(x='Timestamp', y=col, data=df_clean.iloc[::1000])
    plt.title(f'{col} over Time (Downsampled)')
    plt.xlabel('Time')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()



#### Cleaning Impact Analysis


In [ ]:

df_clean['is_outlier'] = outliers  # from earlier Z-score calc
mod_avg = df_clean.groupby('is_outlier')[['ModA', 'ModB']].mean()

mod_avg.plot(kind='bar', figsize=(8, 5))
plt.title('ModA & ModB Averages (Pre vs Post Cleaning)')
plt.ylabel('Average Value')
plt.xticks([0, 1], ['Clean', 'Outlier'], rotation=0)
plt.tight_layout()
plt.show()

#### Correlation & Scatter Analysis

In [ ]:

corr_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'RH', 'WS', 'WSgust']
plt.figure(figsize=(10, 8))
sns.heatmap(df_clean[corr_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()

#### Scatter PLots

In [ ]:

scatter_pairs = [
    ('WS', 'GHI'),
    ('WSgust', 'GHI'),
    ('WD', 'GHI'),
    ('RH', 'Tamb'),
    ('RH', 'GHI')
]

for x, y in scatter_pairs:
    # Ensure both columns are numeric for plotting
    df_plot = df_clean.copy()
    df_plot[x] = pd.to_numeric(df_plot[x], errors='coerce')
    df_plot[y] = pd.to_numeric(df_plot[y], errors='coerce')
    
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=x, y=y, data=df_plot, alpha=0.5)
    plt.title(f'{x} vs {y}')
    plt.tight_layout()
    plt.show()

#### Wind & Distribution Analysis

In [ ]:

# Histogram
df_clean[['GHI', 'WS']].hist(bins=50, figsize=(12, 5))
plt.suptitle('Histograms of GHI and WS')
plt.tight_layout()
plt.show()

In [ ]:
%pip install nbformat

# Wind rose plot using Plotly
fig = px.scatter_polar(df_clean, r='WS', theta='WD', color='Location',
                       title='Wind Rose (Speed vs Direction)', opacity=0.6)
fig.show()


#### Temperature & RH Influence

In [ ]:

# Ensuring 'RH' is numeric and handle "NAN" strings
df_clean['RH'] = pd.to_numeric(df_clean['RH'], errors='coerce')

plt.figure(figsize=(6, 4))
sns.scatterplot(x='RH', y='Tamb', data=df_clean, alpha=0.5)
plt.title('Relative Humidity vs Ambient Temperature')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.scatterplot(x='RH', y='GHI', data=df_clean, alpha=0.5)
plt.title('Relative Humidity vs Global Horizontal Irradiance')
plt.tight_layout()
plt.show()

#### Bubble Chart

In [ ]:

# Remove rows with NaN in 'Tamb', 'GHI', or 'RH' for plotting
plot_df = df_clean.dropna(subset=['Tamb', 'GHI', 'RH'])

fig = px.scatter(plot_df, x='Tamb', y='GHI', size='RH', color='Location',
                 title='GHI vs Tamb (Bubble Size = RH)',
                 labels={'Tamb': 'Ambient Temp', 'GHI': 'Global Horizontal Irradiance'},
                 opacity=0.6)
fig.show()
